# Shen-Atlas regionswise RSA within groups - anonymized version
### model: behavior_delta - positive and negative mood, depressive symptoms and emotion recognition performance
### Note: code cannot be run as a whole - the desired variables to be compared have to be selected manually by running the respective cells
#### import modules & correct read-in (+split groups)

In [ ]:
import os
from os import system as oss
import pandas as pd
import numpy as np
import seaborn as sns
#from matplotlib import pyplot as plt

from sklearn.metrics import pairwise_distances
#from sklearn import metrics
import scipy
from scipy import stats
from scipy.stats import permutation_test
import math

## Modelling - creating dataframes
### Parcel-wise Resting State Connectivity dataframe

In [ ]:
#### Read in Change in Resting State Connectivity Dataframe 
### HDFstore
DF = pd.read_hdf('ParcelwiseConnectivityDelta_Shen268_anonymized.hdf')

### Run one dataframe to be compared (e.g. positive mood or negative mood or depression or emotion recognition) with resting state connectivity
#### Load Itemwise positive mood data

In [ ]:
##read in behav data - Positive Mood
df_behav_neworder = pd.read_excel('HormonesBehaviorDelta_anonymized.xlsx', sheet_name = 'PANASpos_DeltaperItem', index_col = 0)

#### Load Itemwise negative mood data 

In [ ]:
##read in behav data - Negative Mood
df_behav_neworder = pd.read_excel('HormonesBehaviorDelta_anonymized.xlsx', sheet_name = 'PANASneg_DeltaperItem', index_col = 0)

#### Load Itemwise depression score

In [ ]:
##read in behav data - Depressive Symptoms
df_behav_neworder = pd.read_excel('HormonesBehaviorDelta_anonymized.xlsx', sheet_name = 'BDI_DeltaperItem', index_col = 0)

#### Load Itemwise emotion recognition accuracy

In [ ]:
##read in behav data - Emotion Recognition Accuracy
df_behav_neworder = pd.read_excel('HormonesBehaviorDelta_anonymized.xlsx', sheet_name = 'EmoRecogAcc_DeltaperItem', index_col = 0)

#### Load Itemwise emotion recognition response times 

In [ ]:
##read in behav data - Emotion Recognition Response Times
df_behav_neworder = pd.read_excel('HormonesBehaviorDelta_anonymized.xlsx', sheet_name = 'EmoRecogRT_DeltaperEmotion', index_col = 0)

In [ ]:
df_behav_neworder

In [ ]:
DF

### Drop participants with missing behavioral data from both dataframes 

In [ ]:
#reduce data_behav to have the same participants as in RSA by excluding missing data
SubjectswithNAN_behav = df_behav_neworder.loc[pd.isna(df_behav_neworder).any(1), :].index
df_behavFIN = df_behav_neworder.drop(index=SubjectswithNAN_behav)
#make sure that participants without behavioral data are also dropped from RS dataframe
DF_fin = DF.drop(index=SubjectswithNAN_behav)        
# compare whether both dataframes include the same participants 
if DF_fin.index.equals(df_behavFIN.index):
    print("dataframes have the same order and number of participants")
else:
    print("needs checking") 
#len(df_hormonesdeltaFIN)

In [ ]:
df_behavFIN

In [ ]:
DF_fin
#len(DF_fin)

## Regionwise RSA approach - creating and comparing RDMs

### Create RDM for behavioral change 

In [ ]:
### compute RDM for behavioral changes using standardized Euclidean distance
rdm_modelFull = pairwise_distances(df_behavFIN.to_numpy(), metric='seuclidean')
DF_rdm_modelFull = pd.DataFrame(rdm_modelFull)
# vectorize
rdm_modelFull_vec = DF_rdm_modelFull.to_numpy()[np.triu_indices(len(DF_rdm_modelFull.to_numpy()[0]), 1)]

###  Visualize behavioral RDM

In [ ]:
#mask dataframe
mask = np.zeros_like(DF_rdm_modelFull, dtype=bool)
mask[np.triu_indices_from(mask)] = True


#ax = sns.heatmap(DF_rdm_hormone_visualize, mask=mask, yticklabels=False, xticklabels=False, cbar=False, vmin = 0, vmax = 10)
ax = sns.heatmap(DF_rdm_modelFull, mask=mask, yticklabels=False, xticklabels=False, cbar=False, square=True)
# use matplotlib.colorbar.Colorbar object
#cbar = ax.collections[0].colorbar
# here set the labelsize by 20
#cbar.ax.tick_params(labelsize=20)
#plt.show()
#make background transparent
#cbar.patch.set_alpha(0)
ax.patch.set_alpha(0)

print(DF_rdm_modelFull.shape)

# save RDM figure, if needed
#plt.savefig('C:/Users/UKPP/Documents/HormonesRestingStateRSA/ManuscriptRSA_posmoodRDM_standEuclidean.jpg', bbox_inches='tight', dpi=300)

#get info of min and max values as well as matrix size
x = DF_rdm_modelFull.to_numpy()
print(np.max(x[np.nonzero(x)]))
print(np.min(x[np.nonzero(x)]))

### Create RDMs per parcel - functions and loop over all parcels

In [ ]:
### define function to get features of one parcel for one participant
def comp_parcel_feat(all_feat,parcel):
    '''
    get vector (represents connectivity of chosen parcel to every other parcel) for one participant
    '''
    n = len(all_feat)
    N = (math.sqrt(8*n+1)+1)/2
    sq = np.zeros((int(N),int(N)))
    zw = np.ones((int(N),int(N)))
    zw2 = np.tril(zw, k=-1)
    i = np.where(zw2 == 1)
    i_arr = np.asarray(i)
    for nb in range(n):
        sq[i_arr[0,nb],i_arr[1,nb]] = all_feat[nb]       
    sq = sq + sq.T
    parcel_feat = sq[parcel,:]
    features = np.delete(parcel_feat, parcel)
    return features


In [ ]:
##########################################################################################
# Regionswise approach
'''
the code below gets the connectivity per region and calculates the RSA between the model 
and the regionwise connectivity. 
'''
##############################

####### prepare looop
# vessel
all_parcs = {}
all_parcsDF = {}
all_parcsDF_resUncor = {}
all_parcsDF_resPerm = {} 
all_parcsDF_RDM = {}
temp_rdm_df_vec = {}

### loop over parcel
'''
the following double-loop loops over parcels and participants to create
a) parselwise RDM 
b) parcelwise result (RSA distance to model, defined above)
'''
# definitions
parc = []
parcel_ID = range(0, 268) # vector with parcel numbers 268
# loop over parcelID
for parc in parcel_ID:
    print('- parcel: {ID}, loop over parcels (step 1/4) -'.format(ID=parc))
    all_parcs[str(parc)] = {}
# loop over subjects in DF_fin
    for subj in DF_fin.index.to_list():
        all_parcs[str(parc)][subj] = comp_parcel_feat(DF_fin.loc[subj].to_numpy(), parc)
# when done looping over subjects, make DFs out of the sub-dictionary
    print('-- parcel: {ID}, save parcel connectivity as DF in vessel (step 2/4)'.format(ID=parc))
    all_parcsDF[str(parc)] = pd.DataFrame.from_dict(all_parcs[str(parc)]).T
# compute and save RDM for distinct parcels
    print('-- parcel: {ID}, compute and save RDM (step 3/4)'.format(ID=parc))
    temp_rdm = pairwise_distances(all_parcsDF[str(parc)].to_numpy(), metric='euclidean'); 
    all_parcsDF_RDM[str(parc)] = pd.DataFrame(temp_rdm)
# vectorize parcelRDM
    temp_rdm_df_vec[str(parc)] = all_parcsDF_RDM[str(parc)].to_numpy()[np.triu_indices(len(all_parcsDF_RDM[str(parc)].to_numpy()[0]), 1)]
                                                       

In [ ]:
all_parcsDF[str(24)]

### Visualize parcelwise RSFC RDMs 

In [ ]:
# Definitions
parc = []
parcel_ID = [24]  # Vector with example parcel numbers

# Loop over parcel_ID
for parc in parcel_ID:
    # Mask dataframe
    mask = np.zeros_like(all_parcsDF_RDM[str(parc)], dtype=bool)
    mask[np.triu_indices_from(mask)] = True

    # Plot heatmap with masked upper triangle and diagonal
    ax = sns.heatmap(all_parcsDF_RDM[str(parc)], mask=mask, yticklabels=False, xticklabels=False, cbar=False, square=True)

    # Make background transparent
    ax.patch.set_alpha(0)

    print(all_parcsDF_RDM[str(parc)].shape)

    # Save figure if needed
    # plt.savefig('ManuscriptRSA_BrainRDM_Euclidean_parcel{value}_nolabels.jpg'.format(value=parc+1), bbox_inches='tight', dpi=300)

plt.show()

## RSA - with permutation testing (10'000 permutations)

**Permutation Testing**

Using permutation testing to evaluate the significance of RSA analyses (Spearman correlations) via Family-Wise Error (FWE) correction

For each of the 268 parcels: 

(1) subject labels (i.e., rows and columns) are randomly reordered/permuted for one of the two similarity matrices (behavioral RDM) a large number of times (in this case, 10'000)

(2) the correlation between the two matrices (permuted behavioral RDM and original brain RDM) is calculated - this is done 10'000 times - and this forms a null distribution of 10'000 surrogate correlation values

(3) Create a family-wise null distribution of 10'000 values by pooling the 268 null distributions and taking the maximum value for each of the 10'000 positions of the 268 null distributions 

(4) the true correlation values (between the original behavioral RDM and the original brain RDM) is calculated and the observed correlation coefficient is then compared to this family-wise null distribution to obtain a new p-value (assessed at a 0.05 significance threshold, as the FWE correction is done by assessing the ture correlation against a family-wise null distribution


In [ ]:
  ### define function to calculate location of true correlation coefficient in the winner null distribution
def calc_pvalue(null_distr, value):
    
    ''' 
    Function that calculates the p value of a given value on a self made null distribution, i.e. the probability that 
    that value is observed if the null hypothesis is true
    
    null_distr is a list
    value needs to be a single value (so careful when computing correlation, need to feed in only the correlation coefficent and exclude p-value)
    
    '''
    
    # Compute the percentile rank of a score relative to a array of scores (here it's the scores making up the null distribution)
    percentile = scipy.stats.percentileofscore(np.asarray(null_distr), value, kind = 'rank')
    #print(percentile)
    formatted_percentile = "{:.16f}".format(percentile)
    #print(formatted_percentile)
    
    # The corresponding p-value
    p_value = (100 - percentile)/100
    #print(p_value)
    formatted_p_value = "{:.16f}".format(p_value)
    #print(formatted_p_value)
    
    return p_value

### Create null distributions and test true statistic against null distribution of 'winners' across the parcels

In [ ]:
####Permutation testing - same permutation for all parcels 10000 times and then take for each the 'winner'
perm_num = 10000
parcel_ID = range(0, 268)
parc = [] # empty from previous cells

# vessel for the family-wise null distribution of correlation coefficients 
family_wise_null_distr_behav = []

## this is not necessary as the family-wise null distribution will be created in the loop already, but this will be used to later re-build the 268 null distributions by parcel (to check that code worked according to plan)
# creating vessel that will each contain a list of lists representing r values for a given permutation
# called "to transpose" because we will want to make it an array and then .T the array so that it represents a list of lists representing the null distributions made across 268 conditions 
list_null_distr_to_transpose = []


for i in range(perm_num):
        
    ## Creating the permuted behavioral RDM
    # creating an array of weights ('factors') to reorder the rows and columns of the behavioral/hormonal RDM
    perm_fact_behav = np.random.permutation(np.eye(len(DF_rdm_modelFull),dtype=int))
    # creating the reordered (permutated) behavioral matrix based on the original RDM and the above created factors
    df_behav_corrected_perm = (perm_fact_behav @ DF_rdm_modelFull) @ (np.linalg.inv(perm_fact_behav))
    # making distance vector out of that permutated behavioral RDM that will be used to run the RSA - i.e. that will be correlated to the distance vector of the brain RDM
    behav_corrected_perm_vec = df_behav_corrected_perm.to_numpy()[np.triu_indices(len(df_behav_corrected_perm.to_numpy()[0]), 1)]
    # creating vessels that will contain the values constituting the r coefficients of all 268 parcels for a given permutation
    temp_r_coef_for_one_perm_behav = []

    # loop over parcelID
    for parc in parcel_ID:
        # RSA: compute the correlation between the reordered behavioral RDM (vector) and the non-reordered brain RDM (vector)
        corr_behav = scipy.stats.spearmanr(behav_corrected_perm_vec, temp_rdm_df_vec[str(parc)]) 
        # append the correlation coefficient ([0] of corr) to the list that will contain the values making the null distribution of surrogate correlation values 
        temp_r_coef_for_one_perm_behav.append(corr_behav[0])

    # retrieve the max R from this permutation and append to overall array of max values --> only if multiple conditions
    temp_max_value = np.max(temp_r_coef_for_one_perm_behav)
    family_wise_null_distr_behav.append(temp_max_value) 
    
    ### Unnecessary step to later build the 268 null distributions by condition/parcel
    list_null_distr_to_transpose.append(temp_r_coef_for_one_perm_behav)

print('permutations done') 

# vessel for RSA results
RSA_correlation_all_res = {}  # a dictionary for all Spearman correlations of connectome RDMs with behav RDM
RSA_correlation_sig_res = {}  # a dictionary for ONLY statistically significant Spearman correlations of connectome RDMs with behav RDM (at the FWE-corrected 0.05 threshold)

for parc in parcel_ID:           
    # compute and save correlation results (RSA between parcelwiseConnectivity RDM and group-model RDM)
    print('-- parcel: {ID}, statistic calculation(step 4/4)'.format(ID=parc))
    # statistic & save as value in keyParcel
    all_parcsDF_resUncor[str(parc)] = scipy.stats.spearmanr(temp_rdm_df_vec[str(parc)], rdm_modelFull_vec)
    ## permutation testing correction
    # the key of the dictionary has another dictionary as value
    RSA_correlation_all_res[str(parc)] = {} 
    RSA_correlation_all_res[str(parc)]['corr_coef'] = all_parcsDF_resUncor[str(parc)][0]
    RSA_correlation_all_res[str(parc)]['dist_coeff'] = 1 - all_parcsDF_resUncor[str(parc)][0]
    RSA_correlation_all_res[str(parc)]['p-value'] = calc_pvalue(family_wise_null_distr_behav, all_parcsDF_resUncor[str(parc)][0])
    ##Making dictionary for only statistically significant (FWE-corrected p < 0.05 level)
    if RSA_correlation_all_res[str(parc)]['p-value'] <= 0.05:
        # the key of the dictionary has another dictionary as value
        RSA_correlation_sig_res[str(parc)] = {}  
        RSA_correlation_sig_res[str(parc)]['corr_coef'] = all_parcsDF_resUncor[str(parc)][0]
        RSA_correlation_sig_res[str(parc)]['dist_coef'] = 1 - all_parcsDF_resUncor[str(parc)][0]
        RSA_correlation_sig_res[str(parc)]['p-value'] = RSA_correlation_all_res[str(parc)]['p-value']
print('done')
#
    

### Save results of Positive Mood-RSFC RSA 

In [ ]:
### save results for depression score RSA
# File paths
file1_path = "outputdelta_posmood_seuclidean_twotaileduncorrected.txt"
file2_path = "outputdelta_posmood_seuclidean_twotailedFWEcorrected.txt"

# Remove files if they exist
if os.path.exists(file1_path):
    os.remove(file1_path)
if os.path.exists(file2_path):
    os.remove(file2_path)
    
for key in all_parcsDF_resUncor:
    with open(file1_path, "a") as file1, open(file2_path, "a") as file2:
        file1.write('parcel {key}: {value}\n'.format(key=str(int(key)+1), value=all_parcsDF_resUncor[key]))
        file2.write('parcel {key}: rho {value1}, p-value {value2}\n'.format(key=str(int(key)+1), value1=RSA_correlation_all_res[key]['corr_coef'], value2=RSA_correlation_all_res[key]['p-value']))

### Save results of Negative Mood-RSFC RSA 

In [ ]:
### save results for depression score RSA
# File paths
file1_path = "outputdelta_negmood_seuclidean_twotaileduncorrected.txt"
file2_path = "outputdelta_negmood_seuclidean_twotailedFWEcorrected.txt"

# Remove files if they exist
if os.path.exists(file1_path):
    os.remove(file1_path)
if os.path.exists(file2_path):
    os.remove(file2_path)
    
for key in all_parcsDF_resUncor:
    with open(file1_path, "a") as file1, open(file2_path, "a") as file2:
        file1.write('parcel {key}: {value}\n'.format(key=str(int(key)+1), value=all_parcsDF_resUncor[key]))
        file2.write('parcel {key}: rho {value1}, p-value {value2}\n'.format(key=str(int(key)+1), value1=RSA_correlation_all_res[key]['corr_coef'], value2=RSA_correlation_all_res[key]['p-value']))

### Save results of Depressive Score-RSFC RSA 

In [ ]:
### save results for depression score RSA
# File paths
file1_path = "outputdelta_BDI_seuclidean_twotaileduncorrected.txt"
file2_path = "outputdelta_BDI_seuclidean_twotailedFWEcorrected.txt"

# Remove files if they exist
if os.path.exists(file1_path):
    os.remove(file1_path)
if os.path.exists(file2_path):
    os.remove(file2_path)
    
for key in all_parcsDF_resUncor:
    with open(file1_path, "a") as file1, open(file2_path, "a") as file2:
        file1.write('parcel {key}: {value}\n'.format(key=str(int(key)+1), value=all_parcsDF_resUncor[key]))
        file2.write('parcel {key}: rho {value1}, p-value {value2}\n'.format(key=str(int(key)+1), value1=RSA_correlation_all_res[key]['corr_coef'], value2=RSA_correlation_all_res[key]['p-value']))

### Save results of Emotion Recognition Accuracy-RSFC RSA 

In [ ]:
### save results for depression score RSA
# File paths
file1_path = "outputdelta_emorecacc_seuclidean_twotaileduncorrected.txt"
file2_path = "outputdelta_emorecacc_seuclidean_twotailedFWEcorrected.txt"

# Remove files if they exist
if os.path.exists(file1_path):
    os.remove(file1_path)
if os.path.exists(file2_path):
    os.remove(file2_path)
    
for key in all_parcsDF_resUncor:
    with open(file1_path, "a") as file1, open(file2_path, "a") as file2:
        file1.write('parcel {key}: {value}\n'.format(key=str(int(key)+1), value=all_parcsDF_resUncor[key]))
        file2.write('parcel {key}: rho {value1}, p-value {value2}\n'.format(key=str(int(key)+1), value1=RSA_correlation_all_res[key]['corr_coef'], value2=RSA_correlation_all_res[key]['p-value']))

### Save results of Emotion Recognition Times-RSFC RSA 

In [ ]:
# Save results for Emotion Recognition Response Times
# File paths
file1_path = "outputdelta_emorecrt_seuclidean_twotaileduncorrected.txt"
file2_path = "outputdelta_emorecrt_seuclidean_twotailedFWEcorrected.txt"

# Remove files if they exist
if os.path.exists(file1_path):
    os.remove(file1_path)
if os.path.exists(file2_path):
    os.remove(file2_path)
    
for key in all_parcsDF_resUncor:
    with open(file1_path, "a") as file1, open(file2_path, "a") as file2:
        file1.write('parcel {key}: {value}\n'.format(key=str(int(key)+1), value=all_parcsDF_resUncor[key]))
        file2.write('parcel {key}: rho {value1}, p-value {value2}\n'.format(key=str(int(key)+1), value1=RSA_correlation_all_res[key]['corr_coef'], value2=RSA_correlation_all_res[key]['p-value']))

In [ ]:
key

## Create histrogram of 'winner' null distribution across all 268 parcels

In [ ]:
# histogram of the null distribution of the surrogate Spearman correlation values
plt.hist(family_wise_null_distr_behav, bins=50, alpha = 0.25)
plt.title('Family-wise null distribution of surrogate Spearman correlation values for CogAC network (state)')
plt.grid()
plt.show()
print("Mean:  ", round(np.mean(family_wise_null_distr_behav),3))
print("Median:", round(np.median(family_wise_null_distr_behav),3))
print("Minimum:", round(np.min(family_wise_null_distr_behav),3))
print("Maximum:", round(np.max(family_wise_null_distr_behav),3))


### for investigating all null distributions seperate by parcel

In [ ]:
# contain a list of lists representing the null distributions made across 268 parcels
list_null_distr = np.array(list_null_distr_to_transpose).T
list_null_distr = list_null_distr.tolist()
# dataframe representing the null distributions made across 268 parcels
df_list_null_distr = pd.DataFrame(list_null_distr)
df_list_null_distr